Умпортируем библиотеку, прочитаем файл, выведем результат.

In [50]:
import pandas as pd
data = pd.read_csv('1.txt', delimiter = '\t')
data.head(5)

,login,tid,Microtasks,assigned_ts,closed_ts
0,login0,190563850.0,4.0,2017-04-20 12:09:39,2017-04-20 13:13:01
1,login0,190561754.0,1.0,2017-04-20 12:10:30,2017-04-20 12:28:29
2,login0,190565906.0,4.0,2017-04-20 12:21:31,2017-04-20 13:30:10
3,login0,190560246.0,1.0,2017-04-20 12:28:30,2017-04-20 13:13:56
4,login0,190562168.0,2.0,2017-04-20 12:28:42,2017-04-20 13:14:50


Выведем информацию о данных, хранимых в таблице.

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701827 entries, 0 to 701826
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   login        701827 non-null  object 
 1   tid          701827 non-null  float64
 2   Microtasks   701827 non-null  float64
 3   assigned_ts  701827 non-null  object 
 4   closed_ts    701827 non-null  object 
dtypes: float64(2), object(3)
memory usage: 26.8+ MB


Выведем количество уникальных асессоров.

In [52]:
data['login'].value_counts()

login7      13721
login224    11462
login73     11377
login139     9615
login652     8144
            ...  
login72         1
login25         1
login130        1
login752        1
login282        1
Name: login, Length: 767, dtype: int64

Рассчитаем время работы над заданием, добавим получивщиеся значения в таблицу, удалим повторяющиеся строки.

In [53]:
import datetime

end = pd.to_datetime(data['closed_ts'])
start = pd.to_datetime(data['assigned_ts'])
work_time = pd.Series(pd.to_timedelta(end - start))
work_time = work_time.dt.total_seconds().to_frame()
data['work_time'] = work_time
data = data.drop_duplicates()
data.head(5)

,login,tid,Microtasks,assigned_ts,closed_ts,work_time
0,login0,190563850.0,4.0,2017-04-20 12:09:39,2017-04-20 13:13:01,3802.0
1,login0,190561754.0,1.0,2017-04-20 12:10:30,2017-04-20 12:28:29,1079.0
2,login0,190565906.0,4.0,2017-04-20 12:21:31,2017-04-20 13:30:10,4119.0
3,login0,190560246.0,1.0,2017-04-20 12:28:30,2017-04-20 13:13:56,2726.0
4,login0,190562168.0,2.0,2017-04-20 12:28:42,2017-04-20 13:14:50,2768.0


Проверим время работы на отрицательные значения

In [54]:
data[data['work_time'] < 0]

,login,tid,Microtasks,assigned_ts,closed_ts,work_time
534453,login585,197340894.0,1.0,2017-05-24 16:13:06,2017-05-24 16:00:57,-729.0


Удалим записи со временем работы меньше нуля.

In [55]:
data = data.drop(data[data['work_time'] < 0].index)
data.head(5)

,login,tid,Microtasks,assigned_ts,closed_ts,work_time
0,login0,190563850.0,4.0,2017-04-20 12:09:39,2017-04-20 13:13:01,3802.0
1,login0,190561754.0,1.0,2017-04-20 12:10:30,2017-04-20 12:28:29,1079.0
2,login0,190565906.0,4.0,2017-04-20 12:21:31,2017-04-20 13:30:10,4119.0
3,login0,190560246.0,1.0,2017-04-20 12:28:30,2017-04-20 13:13:56,2726.0
4,login0,190562168.0,2.0,2017-04-20 12:28:42,2017-04-20 13:14:50,2768.0


Посмотрим какое количество заданий, микрозаданий выполнили пользователи и сколько суммарно затратили на это времени.

In [60]:
def summary(x):
    result = {
        'tid_count': x['tid'].count(),
        'mt_sum': x['Microtasks'].sum(),
        'wt_sum': x['work_time'].sum(),
    }
    return pd.Series(result).round(0)

In [61]:
data.groupby(by=['login']).apply(summary).sort_values(by=['wt_sum'], ascending=False).head(5)

,tid_count,mt_sum,wt_sum
login,,,
login12,870.0,5774.0,22707329.0
login300,325.0,503.0,22618461.0
login54,2703.0,15469.0,22248014.0
login403,1330.0,6754.0,21318538.0
login293,1142.0,6521.0,19555048.0


Сгруппируем данные по login и tid.

In [62]:
data.groupby(['login', 'tid']).agg(mt_sum = ('Microtasks', 'sum'), wt_sum = ('work_time', 'sum')).head(10)

mt_sum   wt_sum
login  tid                         
login0 160783976.0    10.0  46337.0
       160783978.0    10.0  47058.0
       162593178.0     5.0    224.0
       162593182.0     2.0    194.0
       162593198.0    10.0    119.0
       162593224.0     4.0   2035.0
       162593230.0     9.0   1389.0
       162593358.0     7.0    218.0
       162593364.0    10.0   1898.0
       162593366.0     9.0   1986.0

Поскольку разные задания имеют разную сложность и, соответственно, время исполнения. Считаю справедливым ввести дифференцированную оплату.
Для её расчёта предлагаю:
1. узнать наиболее часто встречающееся время выполнения каждого задания (f_work_time).
2. расчитать отклонение реально затраченного времени от наиболее часто встречающегося (∆).
3. для каждого задания, в зависимости от сложности ввести коэффициент корректировки k в диапазоне (0; 1]. Чем сложнее задание, тем ближе к 1 коэффициент.

Формулы для расчёта 

∆ =  (work_time-f_work_time)/(work_time)

∆ < 0, cost = (|∆| + 1) * k * N,
∆ = 0, cost = k * N,
∆ > 0, cost = (1 - ∆) * k * N
